In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

In [2]:
crime_df = pd.read_csv('data/SPD_Crime_Data__2008-Present_20240704.csv')

In [3]:
crime_df.columns = crime_df.columns.str.lower().str.strip().str.replace(' ','_')
crime_df['offense_start_datetime'] = pd.to_datetime(crime_df['offense_start_datetime'])
crime_df['report_datetime'] = pd.to_datetime(crime_df['report_datetime'])
crime_df.dropna(subset=['mcpp', 'offense_start_datetime'], inplace=True)
crime_df.drop_duplicates(inplace=True)  
crime_df['offense'] = crime_df['offense'].str.lower()
crime_df['year'] = crime_df['offense_start_datetime'].dt.year
crime_df['month'] = crime_df['offense_start_datetime'].dt.month

In [4]:
def create_yearly_report(df, year_prefix, mcpp_col):
    """
    Creates a DataFrame for the given year prefix with MCPP counts.
    
    :param df: The DataFrame containing the data.
    :param year_prefix: The year prefix to filter on (e.g., '2014-').
    :param mcpp_col: The name of the column that contains the MCPP values.
    :return: A DataFrame with MCPP counts for the specified year.
    """
    filtered_df = df[df['report_number'].astype(str).str.startswith(year_prefix)]
    mcpp_counts = filtered_df[mcpp_col].value_counts()
    report_count = pd.DataFrame(mcpp_counts).T
    report_count.index = [year_prefix.strip('-')]
    return report_count

# List of years to process
years = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

# Initialize an empty DataFrame to hold all results
all_reports_df = pd.DataFrame()

# Process each year and concatenate the results
for year in years:
    year_prefix = f'{year}-'
    yearly_report = create_yearly_report(crime_df, year_prefix, 'mcpp')
    all_reports_df = pd.concat([all_reports_df, yearly_report])

# Display the combined DataFrame
print(all_reports_df)

mcpp  DOWNTOWN COMMERCIAL  CAPITOL HILL  NORTHGATE  QUEEN ANNE  SLU/CASCADE  \
2014                 6205          4640       4068        3291         2848   
2015                 6485          4635       4392        3391         2853   
2016                 6294          4532       4392        3331         3109   
2017                 7041          4786       4745        3540         3130   
2018                 6984          4712       4473        3722         3234   
2019                 6158          4308       3962        3787         3006   
2020                 4082          4471       3977        4453         3295   
2021                 4181          4328       4228        3765         3343   
2022                 4632          4644       4086        4441         3403   

mcpp  UNIVERSITY  ROOSEVELT/RAVENNA  FIRST HILL  BELLTOWN  BALLARD SOUTH  ...  \
2014        2648               2430        2291      2004           1961  ...   
2015        2922               2245        2020

In [5]:
all_reports_df

mcpp,DOWNTOWN COMMERCIAL,CAPITOL HILL,NORTHGATE,QUEEN ANNE,SLU/CASCADE,UNIVERSITY,ROOSEVELT/RAVENNA,FIRST HILL,BELLTOWN,BALLARD SOUTH,...,ALKI,SOUTH DELRIDGE,UNKNOWN,GENESEE,MADISON PARK,EASTLAKE - EAST,PIGEON POINT,COMMERCIAL DUWAMISH,COMMERCIAL HARBOR ISLAND,<Null>
2014,6205,4640,4068,3291,2848,2648,2430,2291,2004,1961,...,313,307,301,260,250,118,62,33,15,NaN
2015,6485,4635,4392,3391,2853,2922,2245,2020,1744,2015,...,309,279,341,229,251,124,51,24,22,NaN
2016,6294,4532,4392,3331,3109,2881,2539,1745,1710,2193,...,363,344,404,177,308,114,64,40,22,NaN
2017,7041,4786,4745,3540,3130,2974,2641,2042,2209,2405,...,399,333,656,143,204,121,68,61,38,NaN
2018,6984,4712,4473,3722,3234,2914,2633,2215,1884,2654,...,309,357,689,168,203,87,84,71,36,NaN
2019,6158,4308,3962,3787,3006,3004,2651,2573,1771,2634,...,296,308,484,138,192,103,84,37,36,NaN
2020,4082,4471,3977,4453,3295,3135,3171,2500,1594,3136,...,407,255,1122,231,331,102,85,41,33,NaN
2021,4181,4328,4228,3765,3343,2979,3002,2450,1686,2821,...,362,286,856,249,225,126,89,78,27,NaN
2022,4632,4644,4086,4441,3403,3152,2993,2386,1722,2669,...,410,369,1295,283,235,138,136,79,42,1.0


In [6]:
def create_summary_by_prefix(df, prefixes, mcpp_col):
    """
    Creates a summary DataFrame where each row represents a year prefix and columns are MCPP counts.

    :param df: The DataFrame containing the data.
    :param prefixes: List of year prefixes to filter on (e.g., ['2014-', '2015-']).
    :param mcpp_col: The name of the column that contains the MCPP values.
    :return: A DataFrame with MCPP counts for each year prefix.
    """
    summary_dfs = []
    
    for prefix in prefixes:
        filtered_df = df[df['report_number'].astype(str).str.startswith(prefix)]
        mcpp_counts = filtered_df[mcpp_col].value_counts()
        report_count = pd.DataFrame(mcpp_counts).T
        report_count.index = [prefix.strip('-')]
        summary_dfs.append(report_count)
    
    # Combine all yearly reports into a single DataFrame
    combined_df = pd.concat(summary_dfs)
    return combined_df

# List of prefixes to process
prefixes = ['2014-', '2015-', '2016-', '2017-', '2018-', '2019-', '2020-', '2021-', '2022-']

# Generate the summary DataFrame
summary_df = create_summary_by_prefix(crime_df, prefixes, 'mcpp')

# Display the combined DataFrame
#print(summary_df)


In [7]:
#changing column names and values
summary_df['SOUTH BEACON HILL/NEWHOLLY'] = summary_df['SOUTH BEACON HILL'] + summary_df['NEW HOLLY']
summary_df['PIONEER SQUARE/INTERNATIONAL DISTRICT'] = summary_df['CHINATOWN/INTERNATIONAL DISTRICT'] + summary_df['PIONEER SQUARE']

In [8]:
summary_df = summary_df.reset_index()

In [9]:
print(summary_df.columns.to_list())
summary_df.reset_index(inplace=True)

['index', 'DOWNTOWN COMMERCIAL', 'CAPITOL HILL', 'NORTHGATE', 'QUEEN ANNE', 'SLU/CASCADE', 'UNIVERSITY', 'ROOSEVELT/RAVENNA', 'FIRST HILL', 'BELLTOWN', 'BALLARD SOUTH', 'LAKECITY', 'CHINATOWN/INTERNATIONAL DISTRICT', 'CENTRAL AREA/SQUIRE PARK', 'GREENWOOD', 'BALLARD NORTH', 'BITTERLAKE', 'SANDPOINT', 'PIONEER SQUARE', 'WALLINGFORD', 'NORTH BEACON HILL', 'FREMONT', 'BRIGHTON/DUNLAP', 'MAGNOLIA', 'ROXHILL/WESTWOOD/ARBOR HEIGHTS', 'MADRONA/LESCHI', 'SODO', 'MOUNT BAKER', 'RAINIER BEACH', 'MID BEACON HILL', 'GEORGETOWN', 'HIGHLAND PARK', 'ALASKA JUNCTION', 'SOUTH PARK', 'RAINIER VIEW', 'JUDKINS PARK/NORTH BEACON HILL', 'NORTH ADMIRAL', 'MONTLAKE/PORTAGE BAY', 'PHINNEY RIDGE', 'MILLER PARK', 'COLUMBIA CITY', 'NEW HOLLY', 'LAKEWOOD/SEWARD PARK', 'EASTLAKE - WEST', 'HIGH POINT', 'MORGAN', 'CLAREMONT/RAINIER VISTA', 'NORTH DELRIDGE', 'HILLMAN CITY', 'SOUTH BEACON HILL', 'FAUNTLEROY SW', 'ALKI', 'SOUTH DELRIDGE', 'UNKNOWN', 'GENESEE', 'MADISON PARK', 'EASTLAKE - EAST', 'PIGEON POINT', 'COMMERCI

In [14]:
summary_df_melt = pd.melt(summary_df,                                        # dataset
                        id_vars=['index'],      # Column(s) to use as identifier variables
                        value_vars= ['DOWNTOWN COMMERCIAL', 'CAPITOL HILL', 'NORTHGATE', 'QUEEN ANNE', 'SLU/CASCADE', 'UNIVERSITY', 'ROOSEVELT/RAVENNA', 'FIRST HILL', 'BELLTOWN', 'BALLARD SOUTH', 'LAKECITY', 'CHINATOWN/INTERNATIONAL DISTRICT', 'CENTRAL AREA/SQUIRE PARK', 'GREENWOOD', 'BALLARD NORTH', 'BITTERLAKE', 'SANDPOINT', 'PIONEER SQUARE', 'WALLINGFORD', 'NORTH BEACON HILL', 'FREMONT', 'BRIGHTON/DUNLAP', 'MAGNOLIA', 'ROXHILL/WESTWOOD/ARBOR HEIGHTS', 'MADRONA/LESCHI', 'SODO', 'MOUNT BAKER', 'RAINIER BEACH', 'MID BEACON HILL', 'GEORGETOWN', 'HIGHLAND PARK', 'ALASKA JUNCTION', 'SOUTH PARK', 'RAINIER VIEW', 'JUDKINS PARK/NORTH BEACON HILL', 'NORTH ADMIRAL', 'MONTLAKE/PORTAGE BAY', 'PHINNEY RIDGE', 'MILLER PARK', 'COLUMBIA CITY', 'NEW HOLLY', 'LAKEWOOD/SEWARD PARK', 'EASTLAKE - WEST', 'HIGH POINT', 'MORGAN', 'CLAREMONT/RAINIER VISTA', 'NORTH DELRIDGE', 'HILLMAN CITY', 'SOUTH BEACON HILL', 'FAUNTLEROY SW', 'ALKI', 'SOUTH DELRIDGE', 'UNKNOWN', 'GENESEE', 'MADISON PARK', 'EASTLAKE - EAST', 'PIGEON POINT', 'COMMERCIAL DUWAMISH', 'COMMERCIAL HARBOR ISLAND', 'SOUTH BEACON HILL/NEWHOLLY', 'PIONEER SQUARE/INTERNATIONAL DISTRICT','<Null>'],                    # Name to use for the ‘variable’ column    
                        var_name='neighbourhood',                     # Name to use for the ‘variable’ column
                        value_name='reporting_nb')                           # Name to use for the ‘value’ column

In [15]:
summary_df_melt = summary_df_melt.dropna()
summary_df_melt['index'] = summary_df_melt['index'].astype(int)

In [16]:
summary_df_melt

,index,neighbourhood,reporting_nb
0,2014,DOWNTOWN COMMERCIAL,6205.0
1,2015,DOWNTOWN COMMERCIAL,6485.0
2,2016,DOWNTOWN COMMERCIAL,6294.0
3,2017,DOWNTOWN COMMERCIAL,7041.0
4,2018,DOWNTOWN COMMERCIAL,6984.0
...,...,...,...
545,2019,PIONEER SQUARE/INTERNATIONAL DISTRICT,3586.0
546,2020,PIONEER SQUARE/INTERNATIONAL DISTRICT,2975.0
547,2021,PIONEER SQUARE/INTERNATIONAL DISTRICT,2899.0
548,2022,PIONEER SQUARE/INTERNATIONAL DISTRICT,3024.0


In [17]:
summary_df_melt.to_csv('repo_summary.csv')

In [18]:
summary_df_melt

,index,neighbourhood,reporting_nb
0,2014,DOWNTOWN COMMERCIAL,6205.0
1,2015,DOWNTOWN COMMERCIAL,6485.0
2,2016,DOWNTOWN COMMERCIAL,6294.0
3,2017,DOWNTOWN COMMERCIAL,7041.0
4,2018,DOWNTOWN COMMERCIAL,6984.0
...,...,...,...
545,2019,PIONEER SQUARE/INTERNATIONAL DISTRICT,3586.0
546,2020,PIONEER SQUARE/INTERNATIONAL DISTRICT,2975.0
547,2021,PIONEER SQUARE/INTERNATIONAL DISTRICT,2899.0
548,2022,PIONEER SQUARE/INTERNATIONAL DISTRICT,3024.0
